In [1]:
import pandas as pd
import datetime
import pymongo
import json
from bson import Binary, Code, json_util, ObjectId
from bson.json_util import dumps, RELAXED_JSON_OPTIONS
import pygeoj
import os
import itertools 
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from us import states


In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.education_data
# collection = db.grad_data_2013

In [3]:
# files
rootdir = './data/4YearGraduation'

# lists defined
districts_list = []
years = ['2018','2015','2014','2016','2017']

# for loop to get all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        districts_list.append(data)
#         districts_list.sort()
districts_list.pop(0)
# districts_list.reverse()
grad_df=[]
#  add all 4 year graduation from 2013 to 2018 to mongod
for (file, year) in zip(districts_list, years):
    data = file
#     defined variables labels
    grad_label = "grad_"+ year
    df_label = grad_label +"_df"
    
#     Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)

#     replace * with 0
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('*','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('*','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('*','0')
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('-','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('-','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('-','0')
    # set previous change to numeric (int)
    df_label['FOUR_YR_GRAD_RATE'] = pd.to_numeric(df_label['FOUR_YR_GRAD_RATE'])
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = pd.to_numeric(df_label['FOUR_YR_ADJ_COHORT_COUNT'])
    df_label['GRADUATED_COUNT'] = pd.to_numeric(df_label['GRADUATED_COUNT'])
    grad_df.append(df_label)
    
frame = pd.concat(grad_df, axis=0, ignore_index=True)

frame = frame.filter(['YEAR','COUNTY_NAME', 'DISTRICT_NAME','SCHOOL_NAME','SUBGROUP','FOUR_YR_ADJ_COHORT_COUNT','FOUR_YR_GRAD_RATE','GRADUATED_COUNT'])
frame['YEAR']=frame['YEAR'].astype(int)

frame = frame.sort_values(by=['YEAR'])
frame

/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,YEAR,COUNTY_NAME,DISTRICT_NAME,SCHOOL_NAME,SUBGROUP,FOUR_YR_ADJ_COHORT_COUNT,FOUR_YR_GRAD_RATE,GRADUATED_COUNT
20992,2014,ESSEX,ESSEX CO VOC-TECH,Districtwide Total,Economically Disadvantaged,407,96.31,392
21127,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,White,0,0.00,0
21128,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,Black,205,80.98,166
21129,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,Hispanic,0,0.00,0
21130,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,Asian,0,0.00,0
21131,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,American Indian,0,0.00,0
21132,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,Native Hawaiian,0,0.00,0
21133,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,Two or More Races,0,0.00,0
21134,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,Limited English Proficiency,0,0.00,0
21135,2014,ESSEX,NEWARK CITY,CENTRAL HIGH SCHOOL,Economically Disadvantaged,177,84.75,150


In [4]:
# files
rootdir = './data/district'

# lists defined
budget_list = []
years = ['2017_2018','2016_2017','2015_2016','2014_2015','2013_2014']

# for loop to get all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        budget_list.append(data)
        budget_list.sort()
        budget_list.reverse()
        

df_budget=[]
#  add all 4 year graduation from 2013 to 2018 to mongod
for (file, year) in zip(budget_list, years):
    data = file
#     defined variables labels
    budget_label = "district_budget"+ year
    df_label = budget_label +"_df"    
    
#     Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)
    df_label = df_label.filter(['YEAR','COUNTY_NAME', 'DISTRICT_NAME','TOTAL'])
    df_label = df_label.rename(index=str, columns={"TOTAL": "TOTAL_BUDGET"})
#     df["a"] = pd.to_numeric(df["a"])
#     df_label["YEAR"] = pd.to_numeric(df_label["YEAR"])
    df_budget.append(df_label)

frame_budget = pd.concat(df_budget, axis=0, ignore_index=True)
frame_budget['YEAR']=frame_budget['YEAR'].astype(int)

frame_budget = frame_budget.sort_values(by=['YEAR'])
frame_budget.head()

,YEAR,COUNTY_NAME,DISTRICT_NAME,TOTAL_BUDGET
2906,2014,WARREN,WHITE TWP,1734369
2522,2014,CAPE MAY,WOODBINE BORO,2948581
2521,2014,CAPE MAY,WILDWOOD CREST BORO,504458
2520,2014,CAPE MAY,WILDWOOD CITY,5169720
2519,2014,CAPE MAY,WEST WILDWOOD,38136


In [5]:
census_data = './data/citylocs_geocodio.csv'
census_data = pd.read_csv(census_data)
census_df = pd.DataFrame(census_data)
census_df.head()
census_df.columns = census_df.columns.str.replace(" ", "_")
census_df.head()

,DISTRICT_NAME,Latitude,Longitude,Street,City,State,County,Zip,Country,Number_of_households,Median_household_income,Median_age,Median_age_male,Median_age_female,"Population_by_minimum_level_of_education/Male:_12th_grade,_no_diploma",Population_by_minimum_level_of_education/Male:_High_school_graduate,Population_by_minimum_level_of_education/Female:_12th_grade_no_diploma,Population_by_minimum_level_of_education/Female:_High_school_graduate
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,N Main St,Woodstown,NJ,Salem County,8098,US,406.0,95938.0,40.7,33.3,43.6,0.000,0.210,0.000,0.193
1,WOODBURY CITY,39.8382,-75.1527,N Broad St,Woodbury,NJ,Gloucester County,8096,US,785.0,33205.0,43.0,41.7,44.3,0.015,0.450,0.000,0.368
2,WOODBRIDGE TWP,40.5576,-74.2846,Amboy Ave,Woodbridge,NJ,Middlesex County,7095,US,462.0,82870.0,41.9,38.7,43.2,0.033,0.496,0.000,0.427
3,WOOD-RIDGE BORO,40.8457,-74.0879,2nd St,Wood Ridge,NJ,Bergen County,7075,US,250.0,0.0,40.0,39.9,43.0,0.000,0.356,0.000,0.407
4,WINSLOW TWP,39.6573,-74.8624,Hall St,Winslow,NJ,Camden County,8095,US,487.0,91932.0,37.8,38.8,35.1,0.000,0.384,0.039,0.376


In [6]:
population_data = './data/census_pop.csv'
population_data = pd.read_csv(population_data)
population_df = pd.DataFrame(population_data)
population_df = population_df.rename(columns = {'Zipcode': 'Zip'})
population_df.head()

,Population,Zip
0,22121.0,1832
1,8295.0,1833
2,6675.0,1834
3,13527.0,1835
4,4547.0,1840


In [7]:
result_df = pd.merge(census_df,
                population_df,
                how='left',
                on='Zip'
                )
result_df

,DISTRICT_NAME,Latitude,Longitude,Street,City,State,County,Zip,Country,Number_of_households,Median_household_income,Median_age,Median_age_male,Median_age_female,"Population_by_minimum_level_of_education/Male:_12th_grade,_no_diploma",Population_by_minimum_level_of_education/Male:_High_school_graduate,Population_by_minimum_level_of_education/Female:_12th_grade_no_diploma,Population_by_minimum_level_of_education/Female:_High_school_graduate,Population
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,N Main St,Woodstown,NJ,Salem County,8098,US,406.0,95938.0,40.7,33.3,43.6,0.000,0.210,0.000,0.193,9129.0
1,WOODBURY CITY,39.8382,-75.1527,N Broad St,Woodbury,NJ,Gloucester County,8096,US,785.0,33205.0,43.0,41.7,44.3,0.015,0.450,0.000,0.368,35116.0
2,WOODBRIDGE TWP,40.5576,-74.2846,Amboy Ave,Woodbridge,NJ,Middlesex County,7095,US,462.0,82870.0,41.9,38.7,43.2,0.033,0.496,0.000,0.427,20722.0
3,WOOD-RIDGE BORO,40.8457,-74.0879,2nd St,Wood Ridge,NJ,Bergen County,7075,US,250.0,0.0,40.0,39.9,43.0,0.000,0.356,0.000,0.407,8071.0
4,WINSLOW TWP,39.6573,-74.8624,Hall St,Winslow,NJ,Camden County,8095,US,487.0,91932.0,37.8,38.8,35.1,0.000,0.384,0.039,0.376,180.0
5,WILLINGBORO TWP,40.0279,-74.8691,E River Dr,Willingboro,NJ,Burlington County,8046,US,703.0,76164.0,36.9,33.8,40.2,0.000,0.288,0.000,0.304,31731.0
6,WILDWOOD CITY,38.9918,-74.8149,New Jersey Ave,Wildwood,NJ,Cape May County,8260,US,660.0,31020.0,45.9,47.2,45.7,0.045,0.556,0.024,0.437,13690.0
7,WESTWOOD REGIONAL,40.9912,-74.0326,Westwood Ave,Westwood,NJ,Bergen County,7675,US,478.0,90854.0,40.1,33.9,47.1,0.000,0.309,0.034,0.220,26540.0
8,WESTFIELD TOWN,40.6590,-74.3474,Harrison Ave,Westfield,NJ,Union County,7090,US,418.0,158810.0,45.3,49.4,43.6,0.000,0.073,0.000,0.104,30599.0
9,WEST ORANGE TOWN,40.7987,-74.2390,Fairmount Ave,West Orange,NJ,Essex County,7052,US,247.0,63164.0,51.1,44.8,56.1,0.056,0.432,0.000,0.461,46386.0


In [8]:


result = frame.join(frame_budget, on='YEAR', how='left', lsuffix='_left', rsuffix='_right')
#     df.rename(index=str, columns={"A": "a", "C": "c"})
result = result.rename(index = str, columns={'DISTRICT_NAME_left' : 'DISTRICT_NAME'})


final_df = pd.merge(result, result_df, how ='left', on = 'DISTRICT_NAME')

final_df = final_df[final_df.GRADUATED_COUNT !=0]
final_df = final_df.drop(columns = ['COUNTY_NAME_right','DISTRICT_NAME_right', 'YEAR_right'])
final_df['Zip'] = pd.to_numeric(final_df['Zip'], errors='coerce')
final_df = final_df.dropna(subset=['Zip'])
final_df['Zip'] = final_df['Zip'].astype(int)

final_df

final_dict = final_df.to_dict('records')
final_dict

collection = db.education_data
# drop collection if duplicate
db.collection.drop()
# add data to collection
collection.insert_many(final_dict)

# df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv')
final_df.to_csv(r'./data/merge_data.csv')

final_df.count()

YEAR_left                                                                 18211
COUNTY_NAME_left                                                          18211
DISTRICT_NAME                                                             18211
SCHOOL_NAME                                                               16465
SUBGROUP                                                                  18211
FOUR_YR_ADJ_COHORT_COUNT                                                  18211
FOUR_YR_GRAD_RATE                                                         18211
GRADUATED_COUNT                                                           18211
TOTAL_BUDGET                                                              18211
Latitude                                                                  18211
Longitude                                                                 18211
Street                                                                    18211
City                                    